# Unit 3

## Automatic Instruction Optimization with DSPy

## Automatic Instruction Optimization with DSPy

Welcome to our lesson on **Automatic Instruction Optimization** with **DSPy**\! In our previous lesson, we explored how Few-Shot Learning optimizers enhance your DSPy programs by automatically selecting and generating examples to include in your prompts. Today, we'll focus on a different approach: **optimizing the actual instructions** in your prompts.

-----

While Few-Shot Learning optimizers focus on providing examples to guide the language model, **Instruction Optimization** optimizers focus on improving the natural language instructions themselves. Instead of asking, "What examples should I show the model?" these optimizers ask, "**How should I phrase my request to the model?**"

This distinction is important because the way you phrase your instructions can **significantly impact the model's performance**, even with the same underlying task. A well-crafted instruction can guide the model to produce better outputs without needing additional examples, or it can work alongside examples to further enhance performance.

DSPy offers two powerful instruction optimizers:

  * **COPRO** (**C**ontrastive **P**rompt **O**ptimization): Generates and refines new instructions for each step in your program, optimizing them through **coordinate ascent**.
  * **MIPROv2** (**M**inimum **I**nstruction **P**rompt **O**ptimization v2): Generates instructions that are aware of both your data and any demonstrations, using **Bayesian Optimization** to efficiently search the space of possible instructions.

These optimizers are valuable when you want to keep your prompts concise (reducing token usage) or when you're working with models that respond better to clear instructions.

-----

## Understanding COPRO (Contrastive Prompt Optimization)

**COPRO** is a powerful technique for automatically improving instructions. The core idea is to generate multiple alternative instructions, evaluate them using your metric, and iteratively refine them to find the best-performing set.

The "**contrastive**" aspect comes from how it learns by comparing instructions that lead to correct outputs with those that fail.

COPRO uses **coordinate ascent** (a form of hill-climbing) to optimize instructions:

1.  **Generate Alternatives:** For each module, COPRO generates multiple alternative instructions.
2.  **Evaluate:** It evaluates each alternative using your metric and training data.
3.  **Select & Refine:** It selects the best-performing instruction and repeats the process for multiple iterations, generating new alternatives based on the current best instructions.

### Key COPRO Parameters

| Parameter | Description | Default |
| :--- | :--- | :--- |
| `prompt_model` | The language model used to generate new instruction candidates. | N/A |
| `metric` | A function that evaluates the performance of your program. | N/A |
| `breadth` | The number of new instruction candidates to generate in each iteration. | 16 |
| `depth` | The number of iterations to run the optimization process. | 2 |
| `init_temperature` | The temperature used when generating new instruction candidates (Higher = more diverse candidates). | 1.0 |
| `verbose` | Whether to print detailed information during optimization. | `False` |

COPRO is effective when you have a clear metric and want to **optimize instructions without relying on examples**.

-----

## Implementing COPRO in DSPy

Implementing COPRO involves configuring the optimizer with key parameters and using it to compile your program.

```python
from dspy.teleprompt import COPRO

# Define evaluation parameters for the compilation phase
eval_kwargs = dict(num_threads=16, display_progress=True, display_table=0)

# Create the COPRO optimizer
copro_teleprompter = COPRO(
    prompt_model=model_to_generate_prompts,  # E.g., dspy.LM('openai/gpt-4')
    metric=your_defined_metric,              # Your evaluation metric function
    breadth=num_new_prompts_generated,       # E.g., 16
    depth=times_to_generate_prompts,         # E.g., 2
    init_temperature=prompt_generation_temperature,  # E.g., 1.0
    verbose=False
)

# Compile your program with the optimizer
compiled_program_optimized_signature = copro_teleprompter.compile(
    your_dspy_program,
    trainset=trainset,
    eval_kwargs=eval_kwargs
)
```

The `compile()` process will iteratively refine the natural language instructions within your program's modules, selecting the signature that performs best on the provided `trainset` as measured by the `metric`.

-----

## Understanding MIPROv2 (Minimum Instruction Prompt Optimization)

**MIPROv2** is a more comprehensive optimizer that can optimize both **instructions and few-shot examples**. It generates instructions that are **data-aware** and **demonstration-aware**, meaning they are tailored to work effectively with the specific examples being used.

MIPROv2 uses **Bayesian Optimization** to efficiently explore the search space, often finding better instructions with fewer evaluations than COPRO's coordinate ascent.

### Key MIPROv2 Parameters

| Parameter | Description |
| :--- | :--- |
| `metric` | A function that evaluates the performance of your program. |
| `auto` | Specifies the optimization intensity: `"light"`, `"medium"`, or `"heavy"`. Lighter settings are faster for experimentation, while heavier settings perform more trials for better results. |
| `max_bootstrapped_demos` | (Used in `compile`) Maximum number of new examples to self-generate. |
| `max_labeled_demos` | (Used in `compile`) Maximum number of examples to use directly from the training set. |

MIPROv2 is particularly effective when you have a **reasonable amount of training data** (e.g., 200+ examples) and want to **optimize both instructions and examples in a unified way**.

-----

## Implementing MIPROv2 in DSPy

MIPROv2 supports both few-shot and zero-shot configurations by adjusting the `compile()` parameters.

### Few-Shot Configuration (Optimizing Instructions + Examples)

```python
from dspy.teleprompt import MIPROv2

# Create the MIPROv2 optimizer
teleprompter = MIPROv2(
    metric=gsm8k_metric,
    auto="light"  # Start with "light" for quick experimentation
)

# Compile the program with few-shot parameters
optimized_program = teleprompter.compile(
    program.deepcopy(),
    trainset=trainset,
    max_bootstrapped_demos=3,  # Generate up to 3 examples
    max_labeled_demos=4,       # Use up to 4 existing examples
    requires_permission_to_run=False,
)
```

### Zero-Shot Configuration (Optimizing Instructions Only)

To run MIPROv2 in a zero-shot mode, simply set both demonstration limits to zero during compilation:

```python
from dspy.teleprompt import MIPROv2

# Create the MIPROv2 optimizer
teleprompter = MIPROv2(
    metric=gsm8k_metric,
    auto="light"
)

# Compile the program in zero-shot mode
optimized_program = teleprompter.compile(
    program.deepcopy(),
    trainset=trainset,
    max_bootstrapped_demos=0,  # No generated examples
    max_labeled_demos=0,       # No examples from training set
    requires_permission_to_run=False,
)
```

-----

## Summary and Practice Preview

| Optimizer | Primary Focus | Optimization Mechanism | Recommended Use Case |
| :--- | :--- | :--- | :--- |
| **COPRO** | Instructions Only | Coordinate Ascent (Hill-Climbing) | Focus purely on optimizing instructions, especially for zero-shot prompts. |
| **MIPROv2** | Instructions + Examples | Bayesian Optimization | Optimizing instructions and few-shot examples jointly, especially with moderate-to-large training sets (200+). |

**Guidelines for Selection:**

  * **Instructions Only:** Use **COPRO**.
  * **Instructions + Examples (Best Overall):** Use **MIPROv2**.
  * **Larger Data / Longer Run:** Use **MIPROv2** with `auto="medium"` or `"heavy"` for potentially better results.

In the upcoming practice exercises, you'll gain hands-on experience by implementing both **COPRO** and **MIPROv2** to see how they affect your program's behavior.

In the next lesson, we'll explore **Automatic Finetuning**, the final optimization category in DSPy, which involves updating the weights of the underlying language model itself.

## Tuning COPRO Parameters for Better Instructions

Now that you understand how COPRO works to optimize instructions, let's experiment with its key parameters! In this exercise, you'll tune the COPRO optimizer to see how different settings affect the instruction generation process.

First, you will implement COPRO for a simple math solver. Then, you'll modify two important parameters that control COPRO's behavior:

The breadth parameter, which determines how many candidate prompts are generated in each iteration.
The init_temperature parameter, which controls how diverse or creative the generated prompts will be.
Your task is to implement and run the optimization with different combinations of these parameters and observe how they affect:

The variety of instructions generated.
The optimization progress shown in the output.
The final performance scores.
This hands-on experience will help you develop an intuition for configuring instruction optimizers effectively in your own projects. By the end, you'll have a better understanding of the trade-offs between exploration (trying many diverse candidates) and exploitation (focusing on refining promising instructions).

```python
import dspy
import os
from dspy.teleprompt import COPRO
from dspy.evaluate import Evaluate
from data import get_trainset, get_testset, get_devset, metric

# Set up a simple language model
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.environ['OPENAI_API_KEY'], api_base=os.environ['OPENAI_BASE_URL'])
dspy.configure(lm=lm)

# Define a simple math problem solver program
class MathSolver(dspy.Module):
    def __init__(self):
        super().__init__()
        self.solver = dspy.Predict("question -> answer")
    
    def forward(self, question):
        return self.solver(question=question)

# Get data
trainset = get_trainset()
testset = get_testset()
devset = get_devset()

# Create the base program
base_program = MathSolver()

# Define evaluation parameters
eval_kwargs = dict(num_threads=4, display_progress=True, display_table=0)


# TODO: Create the COPRO optimizer with specified parameters
copro_teleprompter = COPRO(
    
)

# TODO: Compile the program with the optimizer
optimized_program = None


# Set up the evaluator, which can be re-used in your code.
evaluator = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

# Evaluate the optimized program
score = evaluator(optimized_program, metric=metric)

# TODO: Show results
print(f"Optimized instruction: {optimized_program.solver.signature}")


```


## Optimizing QA with COPRO Parameters

## Comparing Few-Shot and Zero-Shot Optimization

## Balancing Optimization Intensity for Better Results

## Balancing Optimization Intensity for Better Results